[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jlfvindel/SW-KG/blob/main/RDFlib/SPARQLWrapper-Colab.ipynb)

## Consultas con SPARQLWrapper

### Resumen de este cuaderno
+ Se instancian clientes [SPARQLWrapper](https://sparqlwrapper.readthedocs.io/en/latest/main.html) para consultas a datastores externos
+ Las [consultas SPARQL](https://www.w3.org/TR/2013/REC-sparql11-query-20130321/) ejecutadas son todas de tipo SELECT
+ Se solicita el retorno de datos en el formato [JSON estándar para respuestas](https://www.w3.org/TR/sparql11-results-json/) a consultas SPARQL SELECT 
+ Los datos recibidos se reorganizan como un dataframe para su presentación tabular

### Cuatro tipos de consultas
En la [panorámica sobre SPARQL](https://www.w3.org/TR/sparql11-overview/) se describe que consta de un protocolo de acceso a datastores y que es tanto un lenguaje de administración como de consulta. Como lenguaje de consulta facilta 4 tipos de consultas:
+ Las consultas de tipo CONSTRUCT o DESCRIBE devuelven un grafo
+ Las consulta de tipo ASK devuelven un valor booleano
+ Y las consultas de tipo SELECT devuelven un conjunto de resultados: n-tuplas intuitivamente percibidas como filas de una tabla con n columnas etiquetadas.

### Formatos de retorno
+ Las consultas SELECT o ASK devuelven conjuntos de resultados independientes, que se pueden recibir en formato  [JSON](http://www.w3.org/TR/sparql11-results-json/), [XML](http://www.w3.org/TR/rdf-sparql-XMLres/) o [CSV/TSV](http://www.w3.org/TR/sparql11-results-csv-tsv/)
+ Las consultas CONSTRUCT o DESCRIBE devuelven grafos, que se pueden secuenciar en cualquiera de los [formatos para grafos RDF](https://www.w3.org/TR/rdf11-primer/#section-graph-syntax): N3, Turtle, RDF/XML, JSON-LD

### Dos opciones de SPARQLWrapper
Del paquete SPARQLWrapper se utiliza el módulo Wrapper y su clase [SPARQLWrapper]((https://sparqlwrapper.readthedocs.io/en/latest/SPARQLWrapper.Wrapper.html#SPARQLWrapper.Wrapper.SPARQLWrapper)) para configurar todo tipo de conexión y de consulta SPARQL, con todo tipo de formato de retorno.

El paquete SPARQLWrapper contiene también el módulo SmartWrapper, que contiene la clase [SPARQLWrapper2]((https://sparqlwrapper.readthedocs.io/en/latest/SPARQLWrapper.SmartWrapper.html#SPARQLWrapper.SmartWrapper.SPARQLWrapper2)), una versión con sintaxis simplificada porque asume que las consultas son de tipo SELECT y el formato de retorno es JSON.

## Ejecución de la consulta

### Instanciación del cliente de consulta
Se supone que [el paquete SPARQLWrapper](https://sparqlwrapper.readthedocs.io/en/latest/main.html) está instalado. El siguiente código instancia un cliente de consulta SPARQLWrapper dirigido a Wikidata 

In [ ]:
# Comente o descomente conforme esté o no instalado
!pip install SPARQLWrapper

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

servidor = "https://query.wikidata.org/sparql" 
cliente = SPARQLWrapper(servidor)

print(f'Se ha creado un cliente de consulta a {cliente.endpoint}\n \
      que asume por defecto respuesta en formato {cliente.returnFormat}')

Se ha creado un cliente de consulta a https://query.wikidata.org/sparql
       que asume por defecto respuesta en formato xml


Se cambia el formato de respuesta solicitado a JSON.

In [2]:
cliente.setReturnFormat(JSON)
print(f'Formato de retorno fijado a {cliente.returnFormat}.\n \
     ¿Confirma el servidor que lo acepta? : \
     {cliente.supportsReturnFormat(cliente.returnFormat)}')

Formato de retorno fijado a json.
      ¿Confirma el servidor que lo acepta? :      True


### Consulta que se pretende remitir
Se va a almacenar una consulta SPARQL adeduada a Wikidata en la variable `consulta`. El texto de esta consulta se incorpora al objeto `cliente` en el atributo `.queryString` mediante el método `setQuery()`.

En este caso se preguntará a Wikidata por _"cuadros que representan un funeral, con su autor y su imagen si estuviera disponible"_. El diseño de esta consulta se podía haber realizado externamente sobre el interfaz web que facilita Wikidata, como se muestra en [este enlace a la consulta](https://query.wikidata.org/#%231%20Cada%20entidad%20%28%3Fcuadro%29%20perteneciente%20%28P31%29%20a%20la%20clase%20%27pintura%27%20%28Q3305213%29%0A%232%20que%20tenga%20relación%20%27representa%27%20%28P180%29%20con%20%27funeral%27%20%28Q201676%29%20y%0A%233%20que%20tiene%20por%20autor%20%28P170%29%20a%20%3Fautor%0A%234%20Y%2C%20opcionalmente%2C%20que%20tenga%20por%20imagen%20%28P18%29%20a%20%3Fimagen%0A%23%20%28la%20variable%20%3Frepresenta%20se%20fija%20al%20texto%20%22funeral%22%20por%20legilibidad%20de%20resultados%29%0A%0ASELECT%20%3Fcuadro%20%3FcuadroLabel%20%20%3Fautor%20%3FautorLabel%20%3Frepresenta%20%3Fimagen%0AWHERE%20%7B%0A%20%20BIND%28%22funeral%22%20as%20%3Frepresenta%29%0A%20%20%3Fcuadro%20wdt%3AP31%20wd%3AQ3305213%20.%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%231%0A%20%20%3Fcuadro%20wdt%3AP180%20wd%3AQ201676%20.%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%232%0A%20%20%3Fcuadro%20wdt%3AP170%20%3Fautor.%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%233%0A%20%20OPTIONAL%20%7B%3Fcuadro%20wdt%3AP18%20%20%3Fimagen%7D%20%20%20%20%20%20%20%20%20%20%234%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20%0A%20%20%20%20bd%3AserviceParam%20wikibase%3Alanguage%20%22es%2Cen%22.%20%7D%20%20%23etiquetas%20en%20español%20o%20inglés%0A%7D%0A)

Hay que observar que la dirección del interfaz web de consulta de Wikidata es [https://query.wikidata.org](https://query.wikidata.org) pero esta página remite la consulta a SPARQL end-point en [https://query.wikidata.org/sparql](https://query.wikidata.org/sparql), como también hacemos desde nuestro cliente local de consulta.

In [3]:
consulta = '''
SELECT ?cuadro ?cuadroLabel  ?autor ?autorLabel ?representa ?imagen
WHERE {
  BIND("funeral" as ?representa)
  ?cuadro wdt:P31 wd:Q3305213 .
  ?cuadro wdt:P180 wd:Q201676 .
  ?cuadro wdt:P170 ?autor.          
  OPTIONAL {?cuadro wdt:P18  ?imagen}
  SERVICE wikibase:label { 
    bd:serviceParam wikibase:language "es,en". }  #etiquetas en español o inglés
}
'''

cliente.setQuery(consulta)

## Recepción de resultados

### El objeto respuesta
El método [.query()](https://sparqlwrapper.readthedocs.io/en/latest/SPARQLWrapper.Wrapper.html#SPARQLWrapper.Wrapper.SPARQLWrapper.query) ejecuta la consulta. La respuesta recibida se encapsula en un objeto de la clase  [QueryResult](https://sparqlwrapper.readthedocs.io/en/latest/SPARQLWrapper.Wrapper.html#SPARQLWrapper.Wrapper.QueryResult)

In [8]:
# Envío de consulta y recepción de resultados
respuesta_queryresult = cliente.query()

print(f'El objeto que encapsula la respuesta es de tipo\n \
{type(respuesta_queryresult)}')

print(f'El formato de respuesta ha sido finalmente {respuesta_queryresult._get_responseFormat()}')

El objeto que encapsula la respuesta es de tipo
 <class 'SPARQLWrapper.Wrapper.QueryResult'>
El formato de respuesta ha sido finalmente json


### Serialización de la respuesta
QueryResult tiene un método `.convert()` que exporta los datos encapsulados en un formato de salida que [depende del formato en que se recibieron](https://sparqlwrapper.readthedocs.io/en/latest/_modules/SPARQLWrapper/Wrapper.html#QueryResult.convert) los datos. Para resultados recibidos en JSON la exportación tiene la estructura de un diccionario Python.

La respuesta a esta consulta SELECT se solicitó en formato JSON. Otros formatos posibles para una consulta SELECT son XML, CSV o TSV.

La composición de ejecución de consulta y de conversión, `.query().convert()`, obviamente se podía haber ejecutado como `cliente.query().convert()` y también mediante llamada al macro `queryAndConvert()`.

In [9]:
respuesta_dicc = respuesta_queryresult.convert()

print(f'Estructura de los datos exportados\n \
     {type(respuesta_dicc)}')

Estructura de los datos exportados
      <class 'dict'>


La siguiente celda secuencia este diccionario con el método `json.dumps` y lo imprime de forma legible.

In [10]:
import json
print( json.dumps(respuesta_dicc, indent=1) )

{
 "head": {
  "vars": [
   "cuadro",
   "cuadroLabel",
   "autor",
   "autorLabel",
   "representa",
   "imagen"
  ]
 },
 "results": {
  "bindings": [
   {
    "representa": {
     "type": "literal",
     "value": "funeral"
    },
    "cuadro": {
     "type": "uri",
     "value": "http://www.wikidata.org/entity/Q94802"
    },
    "autor": {
     "type": "uri",
     "value": "http://www.wikidata.org/entity/Q301"
    },
    "imagen": {
     "type": "uri",
     "value": "http://commons.wikimedia.org/wiki/Special:FilePath/El%20Greco%20-%20The%20Burial%20of%20the%20Count%20of%20Orgaz.JPG"
    },
    "cuadroLabel": {
     "xml:lang": "es",
     "type": "literal",
     "value": "El entierro del Conde de Orgaz"
    },
    "autorLabel": {
     "xml:lang": "es",
     "type": "literal",
     "value": "El Greco"
    }
   },
   {
    "representa": {
     "type": "literal",
     "value": "funeral"
    },
    "cuadro": {
     "type": "uri",
     "value": "http://www.wikidata.org/entity/Q470541"
    

En esta exportación json del diccionario se reconoce la estructura [estandarizada](https://www.w3.org/TR/sparql11-results-json/) en que cualquier servidor SPARQL debe responder si se solicita retorno en JSON:

```json
{
 'head': 
    {'vars': 
      ['var0', 'var1', ...]
    },
 'results': 
    {'bindings': 
       [
         {
           'var0': {'type':xxx, 'value':xxx, 'xml:lang':xxx}},
           'var1': {'type':xxx, 'value':xxx, 'xml:lang':xxx}}
           ...
         }
         {
           'var0': {'type':xxx, 'value':xxx, 'xml:lang':xxx}},
           'var1': {'type':xxx, 'value':xxx, 'xml:lang':xxx}}
           ...
         }
       ]
    }
}
```

Es decir, `bindings` es una lista de elementos que son diccionarios. Cada uno de estos diccionarios tiene: 
+ una clave (una _var_ de las que se encuentran también en la lista `vars` y que corresponden a las variables que se solicitaron en la cabecera de la consulta SELECT)
+ su correspondiente valor, que es a su vez un diccionario con valores para las claves `value`, `type` y opcionalmente `xml:lang` para esa _var_.

In [11]:
for elem in respuesta_dicc['results']['bindings']:
    for elem_key, elem_dicc in elem.items():
        print(f'* {elem_key}:')
        for elem_dicc_key, elem_dicc_value in elem_dicc.items():
            print(f'  + {elem_dicc_key} --> {elem_dicc_value}' )
    print('----------\n')

* representa:
  + type --> literal
  + value --> funeral
* cuadro:
  + type --> uri
  + value --> http://www.wikidata.org/entity/Q94802
* autor:
  + type --> uri
  + value --> http://www.wikidata.org/entity/Q301
* imagen:
  + type --> uri
  + value --> http://commons.wikimedia.org/wiki/Special:FilePath/El%20Greco%20-%20The%20Burial%20of%20the%20Count%20of%20Orgaz.JPG
* cuadroLabel:
  + xml:lang --> es
  + type --> literal
  + value --> El entierro del Conde de Orgaz
* autorLabel:
  + xml:lang --> es
  + type --> literal
  + value --> El Greco
----------

* representa:
  + type --> literal
  + value --> funeral
* cuadro:
  + type --> uri
  + value --> http://www.wikidata.org/entity/Q470541
* autor:
  + type --> uri
  + value --> http://www.wikidata.org/entity/Q159606
* imagen:
  + type --> uri
  + value --> http://commons.wikimedia.org/wiki/Special:FilePath/Millais%20-%20Das%20Tal%20der%20Stille.jpg
* cuadroLabel:
  + xml:lang --> en
  + type --> literal
  + value --> The Vale of Rest
*

##  Dataframe con las respuestas
Los datos obtenidos se pueden almacenar en algún tipo de base de datos y/o procesar. El paquete `pandas` permite gestionar este tipo de datos tabulares.

In [12]:
import pandas as pd

Pandas facilita [pandas.json_normalize()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) que es un método para exportar datos de un diccionario, por mucho anidamiento que presente, a un DataFrame plano. Como este método trabaja sobre diccionarios y no sobre cadenas de caracteres JSON, lo ejecutaremos sobre `respuesta_dicc` en vez de sobre su volcado secuenciado. Para comprobar que se ha construido el DataFrame, se imprime a continuación un resumen del mismo.

In [13]:
respuesta_df = pd.json_normalize(respuesta_dicc,['results','bindings'])
respuesta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   representa.type       36 non-null     object
 1   representa.value      36 non-null     object
 2   cuadro.type           36 non-null     object
 3   cuadro.value          36 non-null     object
 4   autor.type            36 non-null     object
 5   autor.value           36 non-null     object
 6   imagen.type           30 non-null     object
 7   imagen.value          30 non-null     object
 8   cuadroLabel.xml:lang  28 non-null     object
 9   cuadroLabel.type      36 non-null     object
 10  cuadroLabel.value     36 non-null     object
 11  autorLabel.xml:lang   33 non-null     object
 12  autorLabel.type       36 non-null     object
 13  autorLabel.value      36 non-null     object
dtypes: object(14)
memory usage: 4.1+ KB


La presentación completa del DataFrame, conforme al resumen previo, produce la siguiente tabla.

In [14]:
respuesta_df

,representa.type,representa.value,cuadro.type,cuadro.value,autor.type,autor.value,imagen.type,imagen.value,cuadroLabel.xml:lang,cuadroLabel.type,cuadroLabel.value,autorLabel.xml:lang,autorLabel.type,autorLabel.value
0,literal,funeral,uri,http://www.wikidata.org/entity/Q94802,uri,http://www.wikidata.org/entity/Q301,uri,http://commons.wikimedia.org/wiki/Special:File...,es,literal,El entierro del Conde de Orgaz,es,literal,El Greco
1,literal,funeral,uri,http://www.wikidata.org/entity/Q470541,uri,http://www.wikidata.org/entity/Q159606,uri,http://commons.wikimedia.org/wiki/Special:File...,en,literal,The Vale of Rest,es,literal,John Everett Millais
2,literal,funeral,uri,http://www.wikidata.org/entity/Q2494850,uri,http://www.wikidata.org/entity/Q40599,uri,http://commons.wikimedia.org/wiki/Special:File...,en,literal,The Funeral,es,literal,Édouard Manet
3,literal,funeral,uri,http://www.wikidata.org/entity/Q3850297,uri,http://www.wikidata.org/entity/Q5581,uri,http://commons.wikimedia.org/wiki/Special:File...,en,literal,Martyrdom of the Pilgrims and Funeral of Saint...,es,literal,Vittore Carpaccio
4,literal,funeral,uri,http://www.wikidata.org/entity/Q16687609,uri,http://www.wikidata.org/entity/Q12844811,NaN,NaN,en,literal,The Funeral of Ferdowsi,es,literal,Ghazanfar Khaligov
5,literal,funeral,uri,http://www.wikidata.org/entity/Q17339396,uri,http://www.wikidata.org/entity/Q454678,uri,http://commons.wikimedia.org/wiki/Special:File...,en,literal,The Last Judgement and the burial of the dead,es,literal,Bernard van Orley
6,literal,funeral,uri,http://www.wikidata.org/entity/Q18689517,uri,http://www.wikidata.org/entity/Q2841491,uri,http://commons.wikimedia.org/wiki/Special:File...,es,literal,Los funerales de Atahualpa,es,literal,Luis Montero
7,literal,funeral,uri,http://www.wikidata.org/entity/Q20175647,uri,http://www.wikidata.org/entity/Q20822488,uri,http://commons.wikimedia.org/wiki/Special:File...,en,literal,Infant Funeral Procession,es,literal,William P. Chappel
8,literal,funeral,uri,http://www.wikidata.org/entity/Q20175693,uri,http://www.wikidata.org/entity/Q20822488,uri,http://commons.wikimedia.org/wiki/Special:File...,en,literal,Adult Funeral Procession,es,literal,William P. Chappel
9,literal,funeral,uri,http://www.wikidata.org/entity/Q20188034,uri,http://www.wikidata.org/entity/Q1689515,uri,http://commons.wikimedia.org/wiki/Special:File...,en,literal,The Funeral of Lucretia,es,literal,Master of Marradi
